In [9]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim

In [10]:
# Data Augmentation dan Normalisasi
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Konversi ke grayscale (opsional)
    transforms.Resize((48, 48)),  # Resize gambar (opsional jika dataset sudah 48x48)
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalisasi dengan mean=0.5 dan std=0.5
])

# Path ke folder dataset
train_dir = "dataset/train"
test_dir = "dataset/test"

# Dataset dan DataLoader
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Membagi train_dataset menjadi train dan validation set
train_size = int(0.8 * len(train_dataset))  # 80% untuk training
val_size = len(train_dataset) - train_size  # 20% untuk validation
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# DataLoader untuk train, validation, dan test
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Contoh akses data
for images, labels in train_loader:
    print(f"Images batch shape: {images.shape}")
    print(f"Labels batch shape: {labels.shape}")
    break

Images batch shape: torch.Size([32, 1, 48, 48])
Labels batch shape: torch.Size([32])


In [11]:
# Melihat distribusi label di train dataset
label_count = Counter([train_dataset.targets[idx] for idx in train_subset.indices])
print(f"Distribusi label di train dataset: {label_count}")

# Label mapping (index ke nama kelas)
print(f"Mapping index ke kelas: {train_dataset.classes}")

Distribusi label di train dataset: Counter({3: 5755, 4: 3985, 5: 3869, 2: 3285, 0: 3183, 6: 2545, 1: 345})
Mapping index ke kelas: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [12]:
# Model sederhana
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(32 * 24 * 24, 7)  # 7 adalah jumlah kelas
)

# Optimizer dan loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fungsi untuk menghitung akurasi
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()  # Set model ke evaluasi mode
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Training dan evaluasi
num_epochs = 20
for epoch in range(num_epochs):
    # Training mode
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Evaluasi pada validation set
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    # Hitung akurasi
    train_accuracy = calculate_accuracy(train_loader, model)
    val_accuracy = calculate_accuracy(val_loader, model)

    # Cetak metrik
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")
    print("-" * 50)

Epoch 1/20
Train Loss: 1.6554, Train Accuracy: 0.4363
Val Loss: 1.5384, Val Accuracy: 0.4131
--------------------------------------------------
Epoch 2/20
Train Loss: 1.4434, Train Accuracy: 0.5005
Val Loss: 1.4665, Val Accuracy: 0.4514
--------------------------------------------------
Epoch 3/20
Train Loss: 1.3664, Train Accuracy: 0.5118
Val Loss: 1.4784, Val Accuracy: 0.4502
--------------------------------------------------
Epoch 4/20
Train Loss: 1.3158, Train Accuracy: 0.5377
Val Loss: 1.4748, Val Accuracy: 0.4453
--------------------------------------------------
Epoch 5/20
Train Loss: 1.2753, Train Accuracy: 0.5581
Val Loss: 1.4534, Val Accuracy: 0.4594
--------------------------------------------------
Epoch 6/20
Train Loss: 1.2380, Train Accuracy: 0.5565
Val Loss: 1.4977, Val Accuracy: 0.4476
--------------------------------------------------
Epoch 7/20
Train Loss: 1.2108, Train Accuracy: 0.5878
Val Loss: 1.4526, Val Accuracy: 0.4756
-------------------------------------------

In [13]:
# Evaluasi pada test set
test_accuracy = calculate_accuracy(test_loader, model)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.4811
